# Generate data of rat

Round 1 2023.11.7

Round 2 2024.2.21

In [ ]:
%matplotlib inline

In [ ]:
import time
time.asctime()

In [ ]:
RAT_RAWDATA_PATH='.../128/'
FIGURE_DATA_PATH='.../paper/figure_data_linux'

In [ ]:
import glob
import os

import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

from ksd import KSD

from ksd.utils.noise_utils import extract_rawdata,calc_noise,butter_lowpass_filter,notch_filter

querystr='(group=="good")'

get_tp=lambda rat:metadata[metadata.curated&(metadata.rat==rat)].timepoint_int.values

KSD.__version__

In [ ]:
os.chdir(FIGURE_DATA_PATH)
FIGURE_DATA_PATH

## Import, Label and Run

### Metadata generation

In [ ]:
folders=[]
folders.extend(glob.glob(os.path.join(RAT_RAWDATA_PATH,'rat????/week*')))
folders.extend(glob.glob(os.path.join(RAT_RAWDATA_PATH,'rat????/acute')))

metadata=pd.DataFrame(data={'directory':folders})

metadata['hasrhdfile']=metadata.directory.apply(lambda x: glob.glob(os.path.join(x,'*.rhd'))).apply(lambda x: len(x))

metadata['duplicated']=metadata.hasrhdfile.apply(lambda x:x>1)

metadata['converted']=metadata.directory.apply(lambda x: glob.glob(os.path.join(x,'*.bin'))).apply(lambda x: len(x)==1)

metadata['rat']=metadata.directory.apply(lambda x:x.split('/')[-2])

metadata['timepoint']=metadata.directory.apply(lambda x:x.split('/')[-1])

metadata['timepoint_int']=metadata.timepoint.apply(lambda x: 0 if x == 'acute' else eval(x[4:]))

metadata['rhdfilename']=metadata.apply(lambda x:os.path.basename(glob.glob(os.path.join(x.directory,'*.rhd'))[0]) if x.hasrhdfile else None ,axis=1)

metadata['binfilename']=metadata.apply(lambda x:'%s_%s_%s.bin'%(x.rhdfilename[-17:-4],x.rat,x.timepoint) if x.hasrhdfile else None,axis=1)

metadata['whitened']=metadata.directory.apply(lambda x: glob.glob(os.path.join(x,'temp_wh.dat'))).apply(lambda x: len(x)==1)

metadata['filtered']=metadata.directory.apply(lambda x: glob.glob(os.path.join(x,'temp_filtered.dat'))).apply(lambda x: len(x)==1)

metadata['executed']=metadata.directory.apply(lambda x: os.path.exists(os.path.join(x,'kilosort3')))

metadata['postprocessed']=metadata.directory.apply(lambda x: glob.glob(os.path.join(x,'kilosort3_*'))).apply(lambda x: len(x)==1)

metadata['postprocess_directory']=metadata.directory.apply(lambda x: glob.glob(os.path.join(x,'kilosort3_*'))).apply(lambda x: x[0] if len(x)>0 else None)

metadata['curated']=metadata.postprocess_directory.apply(lambda x: False if x is None else len(glob.glob(os.path.join(x,'cluster_info.tsv')))>0)

metadata['ksd_processed']=\
metadata.postprocess_directory.apply(lambda x: os.path.exists(os.path.join(x,'ksd_v1.5'))if x is not None else False)

metadata.sort_values('timepoint_int',inplace=True)

In [ ]:
rats=metadata[metadata.curated].rat.unique()
# rats=['rat0708', 'rat0401', 'rat0317', 'rat0529', 'rat0514', 'rat0324']
rats

In [ ]:
def default_dump(obj): # https://blog.csdn.net/weixin_39561473/article/details/123227500
    """Convert numpy classes to JSON serializable objects."""
    if isinstance(obj, (np.integer, np.floating, np.bool_)):
        return obj.item()
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

### Batch labelling (except for rat0317)

In [ ]:
# metadata[(metadata.curated)&(~metadata.ksd_processed)]

In [ ]:
# for d in metadata.loc[metadata.curated&(metadata.rat!='rat0317'),'postprocess_directory']:
# # for d in metadata.loc[metadata.curated&(metadata.rat=='rat0708')&(metadata.timepoint_int==4),'postprocess_directory']:

#     print(d)
#     cluster_info=pd.read_csv(os.path.join(d,'cluster_info.tsv'),sep='\t')
#     cluster_info.loc[(cluster_info.group!='noise')&(cluster_info.imp<5)&(cluster_info.fr>0.03)&(cluster_info.isi==0),'group']='good'
#     cluster_info.loc[(cluster_info.group!='noise')&(cluster_info.imp<5)&(cluster_info.fr>0.03)&(cluster_info.isi>0),'group']='mua'
#     cluster_info.loc[pd.isna(cluster_info.group),'group']='unsorted'
#     cluster_info.to_csv(os.path.join(d,'cluster_info.tsv'),sep='\t',index=False)

#     cluster_info[['cluster_id','group']].to_csv(os.path.join(d,'cluster_group.tsv'),sep='\t',index=False)

### Batch run KSD (and save)

In [ ]:
# querystr='(group=="good")'
# querystr='(group=="good")|(group=="mua")'

In [ ]:
def run_ksd(d):
    print(d)
    ksd_instance=KSD(
        d,
        phy_subset_dir=os.path.join(d,'subset_onlyfilter'),
        querystr='(group=="good")|(group=="mua")',
        load_subset=True,
        load_if_exists=False,
        mmap_mode=None,
        subfolder='ksd_v1.5',
        imp_threshold=5
    )
    ksd_instance.save()

In [ ]:
# %%time
# rats=['rat1203','rat0415','rat0325','rat0401']
# rats=['rat0317']
# for rat in rats:
#     metadata[metadata.curated&(metadata.rat==rat)&(~metadata.ksd_processed)]\
#     .postprocess_directory.apply(run_ksd)   

### Batch run KSD (without saving)

In [ ]:
def return_ksd_instance(d,q,load_subset=False):
    print(d)
    ksd_instance=KSD(
        d,
        phy_subset_dir=os.path.join(d,'subset_onlyfilter'),
        querystr=q,
        load_subset=load_subset,
        imp_threshold=5, # for rats, determined on 2022.10.22
        subfolder='ksd_v1.5', # 2022.11.5
        mmap_mode=None,
    )
    ksd_instance.run()
    return ksd_instance

In [ ]:
rats=['rat0529', 'rat0514', 'rat0401', 'rat0317', 'rat0708', 'rat0324']

In [ ]:
%%time
# metadata['ksd_all']=None
# metadata.loc[metadata.curated,'ksd_all']=metadata[metadata.curated].postprocess_directory.apply(
#     return_ksd_instance,args=(None,))
# metadata['ksd_good_mua']=None
# metadata.loc[metadata.query('(rat in @rats)&curated').index,'ksd_good_mua']=\
# metadata.query('(rat in @rats)&curated').postprocess_directory.apply(
#     return_ksd_instance,args=('((group=="good")|(group=="mua"))&(fr>0.1)',False))
metadata['ksd_good']=None
metadata.loc[metadata.query('(rat in @rats)&curated').index,'ksd_good']=\
metadata.query('(rat in @rats)&curated').postprocess_directory.apply(
    return_ksd_instance,args=('(group=="good")&(fr>0.1)',False))
# metadata['ksd_mua']=None
# metadata.loc[metadata.query('(rat in @rats)&curated').index,'ksd_mua']=\
# metadata.query('(rat in @rats)&curated').postprocess_directory.apply(
#     return_ksd_instance,args=('(group=="mua")&(fr>0.1)',False))

---

<h1><center><b><font size=6>Analysis part below</font></b></center></h1>

## Figure 2 Upper Part

### Data selection

In [ ]:
metadata['good_cluster_count']=metadata.ksd_good.apply(lambda x:None if x is None else x.cluster_id.__len__())

fig2_metadata=metadata.query(
    '((rat=="rat0324")&(timepoint=="week16"))|'\
    '((rat=="rat0401")&(timepoint=="week11"))|'\
    '((rat=="rat0514")&(timepoint=="week7"))|'\
    '((rat=="rat0529")&(timepoint=="week12"))|'\
    '((rat=="rat0708")&(timepoint=="week11"))|'\
    '((rat=="rat0317")&(timepoint=="week37"))')\
        .copy()
fig2l_metadata=fig2_metadata.copy()

fig2l_metadata.index=range(len(fig2l_metadata))

new_order=np.zeros(6,dtype=int)
for n,i in enumerate([fig2l_metadata.query('rat==@rat').index.values[0] 
     for rat in ["rat0324","rat0401","rat0514","rat0529","rat0708","rat0317"]]):
    new_order[n]=i

fig2l_metadata = fig2l_metadata.reindex(new_order)


fig2l_metadata['rat_index']=['rat%d'%i for i in range(1,len(fig2l_metadata)+1)]

fig2u_metadata=fig2l_metadata.head(5).copy()
fig2l_metadata[['rat_index','rat','timepoint','good_cluster_count']]

### Figure 2c

#### LFP & Raw Trace

In [ ]:
r0708w11_ap=extract_rawdata(
    metadata.query('rat=="rat0708"&timepoint=="week11"').directory.values[0]+'/temp_filtered.dat',
    skip=30,
    window=2
)

In [ ]:
r0708w11_raw=extract_rawdata(
    metadata.query('rat=="rat0708"&timepoint=="week11"').directory.values[0]+'/220919_214327_rat0708_week11.bin',
    skip=30,
    window=2
)

In [ ]:
r0708w11_lfp=butter_lowpass_filter(r0708w11_raw,300,20000)

In [ ]:
r0708w11_lfp=notch_filter(r0708w11_lfp,20000)

In [ ]:
lfp_ch=[0,10,20,30,40,50,60,70,79,90,100,109,119]

In [ ]:
np.savetxt('fig2/fig2c_lfp.csv',r0708w11_lfp[20000:30000,lfp_ch],delimiter=',')
np.savetxt('fig2/fig2c_ap.csv',r0708w11_ap[20000:30000,lfp_ch],delimiter=',')

#### Raster Plot

In [ ]:
r0708w11ksd=metadata.query('rat=="rat0708"&timepoint=="week11"').ksd_good.values[0]

In [ ]:
st_dict=[]
for _,(clid,depth,n_spikes) in r0708w11ksd.info[['cluster_id','depth','n_spikes']].iterrows():
    print(clid,end='\t\r')
    st=r0708w11ksd.spike_times[
            (r0708w11ksd.spike_times>31*20000)&\
            (r0708w11ksd.spike_times<31.5*20000)&\
            (r0708w11ksd.spike_clusters==clid)]/20000
    st_dict.append({'cluster_id':clid,'spike_times':st.tolist(),'depth':depth})
pd.DataFrame(st_dict).to_excel('fig2/fig2c_raster_plot.xlsx',index=False)

### Figure 2d

In [ ]:
# compute_rms=lambda x,axis=0:np.sqrt(np.mean(x**2,axis)) # replaced by MAD/0.6745
def calc_snr(x):
    cluster_info=x.ksd_good.info
    rms=x.channel_RMS
    return cluster_info.apply(lambda cl:cl.real_amp/rms[cl.mch],axis=1).values # real_amp_pp before 2022.12.5

In [ ]:
fig2u_metadata['binfilename']=fig2u_metadata.apply(lambda x:glob.glob(os.path.join(x.directory,'*.bin'))[0],axis=1)
fig2u_metadata['channel_RMS']=fig2u_metadata.binfilename\
    .apply(calc_noise)
fig2u_metadata['channel_RMS']=fig2u_metadata['channel_RMS'].apply(lambda x:x.mad_hp.values)
fig2u_metadata['snr']=fig2u_metadata.apply(calc_snr,axis=1)

In [ ]:
efficiencies=[k.efficiency[0] for _,(r,k) in fig2u_metadata[['rat','ksd_good']].iterrows()]
yields=[k.yield_[0] for _,(r,k) in fig2u_metadata[['rat','ksd_good']].iterrows()]
yields_good_mua=[k.yield_[0] for _,(r,k) in fig2u_metadata[['rat','ksd_good_mua']].iterrows()]
yields_mua=[i-j for i,j in zip(yields_good_mua,yields)]
frs=[k.info.fr.values for _,(r,k) in fig2u_metadata[['rat','ksd_good']].iterrows()]

In [ ]:
amplitudes=[k.info.real_amp.values for _,(r,k) in fig2u_metadata[['rat','ksd_good']].iterrows()]
snrs=[k for _,(r,k) in fig2u_metadata[['rat','snr']].iterrows()]
frs=[k.info.fr.values for _,(r,k) in fig2u_metadata[['rat','ksd_good']].iterrows()]
densities=[k.density[0] for _,(r,k) in fig2u_metadata[['rat','ksd_good']].iterrows()]
spreads=[k.spread[0] for _,(r,k) in fig2u_metadata[['rat','ksd_good']].iterrows()]

In [ ]:
fig2d=dict(efficiencies=efficiencies,
           yields=yields,
           yields_good_mua=yields_good_mua,
           yields_mua=yields_mua,
           amplitudes=amplitudes,
           snrs=snrs,
           frs=frs,
           densities=densities,
           spreads=spreads
          )

In [ ]:
with open('fig2/fig2d.json','w') as fp:
    json.dump(fig2d,fp,default=default_dump,indent=4)

### Figure 2e & Extended Data Figure 6a

In [ ]:
fig2u_raw_trace_selected={
    'week41':[0.88],
    # 'week85':[74.8],
    # 'week100':[142.0],
    'week105':[139.4]
}

for n,(tp,tstarts) in enumerate(fig2u_raw_trace_selected.items()):
    for tstart in tstarts:
        rt=extract_rawdata(RAT_RAWDATA_PATH+'rat0317/%s/temp_filtered.dat'%tp,skip=tstart,window=0.2)
        np.savetxt('fig2/fig2e_raw_trace_%s.csv'%tp,rt,delimiter=',')

In [ ]:
fig2u_raw_trace_selected={
    'acute':[30.2783,],
    'week16':[241.4658,],
    'week33':[23.6675,]
}

for n,(tp,tstarts) in enumerate(fig2u_raw_trace_selected.items()):
    for tstart in tstarts:
        rt=extract_rawdata(RAT_RAWDATA_PATH+'rat0324/%s/temp_filtered.dat'%tp,skip=tstart,window=0.05)
        np.savetxt('ext/ext6a_raw_trace_%s.csv'%tp,rt,delimiter=',')

### Figure 2f & Extended Data Figure 6b

In [ ]:
fig2f={}
for tp in [0,4,10,12,16,21,25,29,33,37,41,47,52,64,77,85,100,105]:
    ksd_instance=metadata.loc[(metadata.rat=='rat0317')&(metadata.timepoint_int==tp),'ksd_good'].values[0]
    mwf=ksd_instance.mean_waveforms
    mwf_array=[]
    for chid in range(69,85): # 54 - 80 (both included, counting from 1)
        this_chid_mwf_array=[]
        mch_clids=ksd_instance.info.query('mch==@chid').cluster_id.values
        if mch_clids is not None:
            for clid in mch_clids:
                this_chid_mwf_array.append((mwf.get(clid).get(chid)[21:-21]-mwf.get(clid).get(chid)[21]).tolist())
        mwf_array.append(this_chid_mwf_array)
    fig2f.update({str(tp):mwf_array})

In [ ]:
with open('fig2/fig2f.json','w') as fp:
    json.dump(fig2f,fp)

In [ ]:
fig2f={}
for tp in [0,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,33]:
    ksd_instance=metadata.loc[(metadata.rat=='rat0324')&(metadata.timepoint_int==tp),'ksd_good'].values[0]
    mwf=ksd_instance.mean_waveforms
    mwf_array=[]

    for chid in range(48,70):

        this_chid_mwf_array=[]
        mch_clids=ksd_instance.info.query('mch==@chid').cluster_id.values
        if mch_clids is not None:
            for clid in mch_clids:
                this_chid_mwf_array.append((mwf.get(clid).get(chid)[21:-21]-mwf.get(clid).get(chid)[21]).tolist())
        mwf_array.append(this_chid_mwf_array)
    fig2f.update({str(tp):mwf_array})

In [ ]:
with open('ext/ext6b.json','w') as fp:
    json.dump(fig2f,fp)

## Figure 2 Lower Part

### Figure 2j

#### Long-term recording stability

In [ ]:
get_channels_have_signals_count=lambda rat,q:metadata.loc[metadata.curated&(metadata.rat==rat),q]\
.apply(lambda x:x.channels_have_signals_count)
get_cluster_count=lambda rat,q:metadata.loc[metadata.curated&(metadata.rat==rat),q]\
.apply(lambda x:x.cluster_id.__len__())
get_tot_fr=lambda rat,q:metadata.loc[metadata.curated&(metadata.rat==rat),q].apply(lambda x:x.info.fr.sum())
get_fr_values=lambda rat,q:metadata.loc[metadata.curated&(metadata.rat==rat),q].apply(lambda x:x.info.fr.values)
get_imp=lambda rat:metadata.loc[metadata.curated&(metadata.rat==rat),'ksd_good']\
.apply(lambda x:x.imp.query('imp<5').imp.values)
get_amp=lambda rat,q:metadata.loc[metadata.curated&(metadata.rat==rat),q].apply(lambda x:x.info.real_amp.values) # 24.3.29

In [ ]:
fig2j={}
for rat in fig2l_metadata.rat.unique():
    fig2j.update({
        rat:{
            'tp':get_tp(rat),
            'cluster_count':get_cluster_count(rat,'ksd_good').values,
            'tot_fr':get_tot_fr(rat,'ksd_good').values,
            'fr_values':get_fr_values(rat,'ksd_good').values,
            'amp':get_amp(rat,'ksd_good').values,
            'channels_have_signals_count':get_channels_have_signals_count(rat,'ksd_good').values
        }
    })

with open('fig2/fig2j.json','w') as fp:
    json.dump(fig2j,fp,default=default_dump)

#### correlation test

In [ ]:
from scipy.stats import linregress

In [ ]:
skip_samples=0
fig2l_metadata['fr_lr_res']=fig2l_metadata.rat.apply(
    lambda x:linregress(get_tp(x)[skip_samples:]*7,get_tot_fr(x,'ksd_good').apply(np.log10)[skip_samples:],
                        'less'))
fig2l_metadata['nc_lr_res']=fig2l_metadata.rat.apply(
    lambda x:linregress(get_tp(x)[skip_samples:]*7,get_cluster_count(x,'ksd_good').apply(np.log10)[skip_samples:],
                        'less'))
fig2l_metadata['amp_lr_res']=fig2l_metadata.rat.apply(
    lambda x:linregress(get_tp(x)[skip_samples:]*7,get_amp(x,'ksd_good').apply(np.median).apply(np.log10)[skip_samples:],
                        'less'))
fig2l_metadata['imp_lr_res']=fig2l_metadata.rat.apply(
    lambda x:linregress(get_tp(x)[skip_samples:]*7,(get_imp(x)*1e6).apply(np.median).apply(np.log10)[skip_samples:],
                        'less'))
fig2l_metadata['active_ch_lr_res']=fig2l_metadata.rat.apply(
    lambda x:linregress(get_tp(x)[skip_samples:]*7,get_channels_have_signals_count(x,'ksd_good').apply(np.log10)[skip_samples:],
                        'less'))

In [ ]:
fig2l_metadata['fr_slope']=fig2l_metadata['fr_lr_res'].apply(lambda x:x.slope)
fig2l_metadata['fr_pvalue']=fig2l_metadata['fr_lr_res'].apply(lambda x:x.pvalue)
fig2l_metadata['nc_slope']=fig2l_metadata['nc_lr_res'].apply(lambda x:x.slope)
fig2l_metadata['nc_pvalue']=fig2l_metadata['nc_lr_res'].apply(lambda x:x.pvalue)
fig2l_metadata['amp_slope']=fig2l_metadata['amp_lr_res'].apply(lambda x:x.slope)
fig2l_metadata['amp_pvalue']=fig2l_metadata['amp_lr_res'].apply(lambda x:x.pvalue)
fig2l_metadata['imp_slope']=fig2l_metadata['imp_lr_res'].apply(lambda x:x.slope)
fig2l_metadata['imp_pvalue']=fig2l_metadata['imp_lr_res'].apply(lambda x:x.pvalue)
fig2l_metadata['active_ch_slope']=fig2l_metadata['active_ch_lr_res'].apply(lambda x:x.slope)
fig2l_metadata['active_ch_pvalue']=fig2l_metadata['active_ch_lr_res'].apply(lambda x:x.pvalue)

In [ ]:
fig2l_metadata[['rat','rat_index','nc_pvalue','fr_pvalue','amp_pvalue','active_ch_pvalue']]

In [ ]:
fig2l_metadata[['rat','rat_index',*[i+'_slope' for i in ['nc','fr','amp','active_ch']],*[i+'_pvalue' for i in ['nc','fr','amp','active_ch']]]].to_excel('fig2/fig2j_pvalues.xlsx',index=False)

### Figure 2h & Extended Data Figure 6d

In [ ]:
rat0324_metadata=metadata.query('rat=="rat0324"&curated').copy()
rat0324_metadata['binfilename']=rat0324_metadata.apply(lambda x:glob.glob(os.path.join(x.directory,'*.bin'))[0],axis=1)
rat0324_metadata['channel_RMS']=rat0324_metadata.binfilename\
    .apply(calc_noise)
rat0324_metadata['channel_RMS']=rat0324_metadata['channel_RMS'].apply(lambda x:x.mad_hp.values)
rat0324_metadata['snr']=rat0324_metadata.apply(calc_snr,axis=1)

In [ ]:
rat0324_metadata[['timepoint_int','snr']].to_json('ext/ext6d.json')

In [ ]:
rat0317_metadata=metadata.query('rat=="rat0317"&curated').copy()
rat0317_metadata['binfilename']=rat0317_metadata.apply(lambda x:glob.glob(os.path.join(x.directory,'*.bin'))[0],axis=1)
rat0317_metadata['channel_RMS']=rat0317_metadata.binfilename\
    .apply(calc_noise)
rat0317_metadata['channel_RMS']=rat0317_metadata['channel_RMS'].apply(lambda x:x.mad_hp.values)
rat0317_metadata['snr']=rat0317_metadata.apply(calc_snr,axis=1)

In [ ]:
rat0317_metadata[['timepoint_int','snr']].to_json('fig2/fig2h.json')

### Figure 2i & Extended Data Figure 6e

In [ ]:
def get_spike_spatial_dist(ksd_instance):
    d=ksd_instance.cluster_info.groupby('ch').fr.sum()/ksd_instance.cluster_info.groupby('ch').fr.sum().sum()
    matrix=np.zeros(128)
    matrix[d.index]=d.values
    return matrix

In [ ]:
rat0317_metadata['spike_channel_dist']=rat0317_metadata.ksd_good.apply(get_spike_spatial_dist)

In [ ]:
rat0317_metadata['spike_channel_dist'].apply(max).max()

In [ ]:
np.savetxt('fig2/fig2i_tp.csv',rat0317_metadata.timepoint_int.values,delimiter=',')
np.savetxt('fig2/fig2i.csv',np.array(rat0317_metadata['spike_channel_dist'].tolist()),delimiter=',')

In [ ]:
rat0324_metadata['spike_channel_dist']=rat0324_metadata.ksd_good.apply(get_spike_spatial_dist)

In [ ]:
rat0324_metadata['spike_channel_dist'].apply(max).max()

In [ ]:
np.savetxt('ext/ext6e_tp.csv',rat0324_metadata.timepoint_int.values,delimiter=',')
np.savetxt('ext/ext6e.csv',np.array(rat0324_metadata['spike_channel_dist'].tolist()),delimiter=',')

### Figure 2g & Extended Data Figure 6c

In [ ]:
get_amp('rat0317','ksd_good').apply(list).to_excel('fig2/fig2g.xlsx',index=False)

fig2g=pd.read_excel('fig2/fig2g.xlsx')

fig2g['timepoint_int']=rat0317_metadata['timepoint_int'].values

fig2g.rename(columns={'ksd_good':'Amplitude'},inplace=True)

fig2g.to_excel('fig2/fig2g.xlsx',index=False)

In [ ]:
get_amp('rat0324','ksd_good').apply(list).to_excel('ext/ext6c.xlsx',index=False)

fig2g=pd.read_excel('ext/ext6c.xlsx')

fig2g['timepoint_int']=rat0324_metadata['timepoint_int'].values

fig2g.rename(columns={'ksd_good':'Amplitude'},inplace=True)

fig2g.to_excel('ext/ext6c.xlsx',index=False)

## Extended Data Figure 5

In [ ]:
si_spread=pd.DataFrame([{
    'rat':'rat0708',
    'timepoint_int':16,
    'cluster_id':0
},{
    'rat':'rat0708',
    'timepoint_int':6,
    'cluster_id':9
},{
    'rat':'rat0324',
    'timepoint_int':16,
    'cluster_id':233
},{
    'rat':'rat0514',
    'timepoint_int':0,
    'cluster_id':285
},{
    'rat':'rat0514',
    'timepoint_int':0,
    'cluster_id':293
},{
    'rat':'rat0708',
    'timepoint_int':11,
    'cluster_id':338
}])

si_spread['mwf']=si_spread.apply(lambda x:\
                metadata.query('(rat==@x.rat)&(timepoint_int==@x.timepoint_int)').\
                ksd_good.values[0].mean_waveforms.get(x.cluster_id),
                axis=1)

si_spread['wf']=si_spread.apply(lambda x:\
                metadata.query('(rat==@x.rat)&(timepoint_int==@x.timepoint_int)').\
                ksd_good.values[0].extract_waveforms(x.cluster_id),
                axis=1)

si_spread['mwf']=si_spread['mwf'].apply(lambda x: dict([(i,j.tolist()) for (i,j) in x.items()])).apply(lambda x: dict(sorted(x.items())))

si_spread['wf']=si_spread['wf'].apply(lambda x: dict([(i,j.tolist()) for (i,j) in x.items()])).apply(lambda x: dict(sorted(x.items())))

#si_spread.to_json('ext/ext5.json')
si_spread.to_json('ext/ext5.json.new') # this process contains randomness, so the result is not reproducible.

## Descriptive Statistics of Rat Data

In [ ]:
def ravel(l):
    r=[]
    for item in l:
        r.extend(item)
    return np.array(r)

In [ ]:
metadata.groupby('rat').postprocessed.count()

In [ ]:
# fig2l_metadata['all_cluster_count']=fig2l_metadata.ksd_good_mua.apply(lambda x:x.cluster_id.size)
# fig2l_metadata['good']=fig2l_metadata.ksd_good.apply(lambda x:x.cluster_id.size)
# fig2l_metadata['mua']=fig2l_metadata.ksd_mua.apply(lambda x:x.cluster_id.size)
# fig2l_metadata['fr_sum']=fig2l_metadata.ksd_good.apply(lambda x:x.info.fr.sum())
# fig2l_metadata['fr_sum_all']=fig2l_metadata.ksd_good_mua.apply(lambda x:x.info.fr.sum())

# fig2l_metadata['fr_good_mean']=fig2l_metadata.ksd_good.apply(lambda x:x.info.fr.mean())
# fig2l_metadata['fr_mua_mean']=fig2l_metadata.ksd_mua.apply(lambda x:x.info.fr.mean())
# fig2l_metadata['fr_all_mean']=fig2l_metadata.ksd_good_mua.apply(lambda x:x.info.fr.mean())


# np.set_printoptions(precision=2)
# pd.set_option("display.precision", 1) # https://stackoverflow.com/questions/43217916/pandas-data-precision
# fig2l_metadata[['rat','timepoint','good','mua','fr_sum','fr_sum_all','fr_good_mean','fr_mua_mean','fr_all_mean']]
# fig2l_metadata['fr_q25']=fig2l_metadata.ksd_good.apply(lambda x:x.info.fr.quantile(0.25))
# fig2l_metadata['fr_q50']=fig2l_metadata.ksd_good.apply(lambda x:x.info.fr.quantile(0.5))
# fig2l_metadata['fr_q75']=fig2l_metadata.ksd_good.apply(lambda x:x.info.fr.quantile(0.5))
# fig2l_metadata['fr_sum']=fig2l_metadata.ksd_good.apply(lambda x:x.info.fr.sum())
# fig2l_metadata['fr_mean']=fig2l_metadata.ksd_good.apply(lambda x:x.info.fr.mean())

# fig2l_metadata['real_amp_q25']=fig2l_metadata.ksd_good.apply(lambda x:x.info.real_amp.quantile(0.25))
# fig2l_metadata['real_amp_q50']=fig2l_metadata.ksd_good.apply(lambda x:x.info.real_amp.quantile(0.5))
# fig2l_metadata['real_amp_q75']=fig2l_metadata.ksd_good.apply(lambda x:x.info.real_amp.quantile(0.5))
# fig2l_metadata['real_amp_sum']=fig2l_metadata.ksd_good.apply(lambda x:x.info.real_amp.sum())
# fig2l_metadata['real_amp_mean']=fig2l_metadata.ksd_good.apply(lambda x:x.info.real_amp.mean())
# fig2l_metadata['spread_q25']=fig2l_metadata.ksd_good.apply(lambda x:np.quantile(x.spread[0],0.25))
# fig2l_metadata['spread_q50']=fig2l_metadata.ksd_good.apply(lambda x:np.quantile(x.spread[0],0.5))
# fig2l_metadata['spread_q75']=fig2l_metadata.ksd_good.apply(lambda x:np.quantile(x.spread[0],0.75))
# fig2l_metadata['spread_mean']=fig2l_metadata.ksd_good.apply(lambda x:np.mean(x.spread[0]))
# fig2l_metadata['density_q25']=fig2l_metadata.ksd_good.apply(lambda x:np.quantile(x.density[0],0.25))
# fig2l_metadata['density_q50']=fig2l_metadata.ksd_good.apply(lambda x:np.quantile(x.density[0],0.5))
# fig2l_metadata['density_q75']=fig2l_metadata.ksd_good.apply(lambda x:np.quantile(x.density[0],0.75))
# fig2l_metadata['density_mean']=fig2l_metadata.ksd_good.apply(lambda x:np.mean(x.density[0]))
# fig2l_metadata[['rat','rat_index','timepoint', 'timepoint_int', 'good_cluster_count', 'all_cluster_count',
#         'fr_q25', 'fr_q50', 'fr_q75', 'fr_sum', 'fr_mean',
#        'real_amp_q25', 'real_amp_q50', 'real_amp_q75', 'real_amp_sum',
#        'real_amp_mean','spread_q25', 'spread_q50',
#        'spread_q75', 'spread_mean', 'density_q25', 'density_q50',
#        'density_q75', 'density_mean']].to_excel('\\\\jsynas2\\home\\paper figure\\data\\fig2l_metadata.xlsx',index=False)

In [ ]:
min(yields_good_mua),max(yields_good_mua) # (143, 276)

In [ ]:
min(yields),max(yields) # (124, 239)

In [ ]:
np.set_printoptions(precision=2)

In [ ]:
np.array([min(efficiencies),max(efficiencies)]) # array([1.24, 2.54])

In [ ]:
fig2u_all_spread=np.concatenate(fig2u_metadata.ksd_good.apply(lambda x:x.spread[0]).values)
fig2u_all_density=np.concatenate(fig2u_metadata.ksd_good.apply(lambda x:x.density[0]).values)

In [ ]:
fig2u_all_fr=np.concatenate(fig2u_metadata.ksd_good.apply(lambda x:x.info.fr.values).values)

fig2u_all_amp=np.concatenate(fig2u_metadata.ksd_good.apply(lambda x:x.info.real_amp.values).values)

fig2u_all_snr=np.concatenate(fig2u_metadata.snr.values)

In [ ]:
np.set_printoptions(precision=1)

In [ ]:
np.array([np.median(fig2u_all_amp),np.quantile(fig2u_all_amp,0.25),np.quantile(fig2u_all_amp,0.75)])
# array([67.9, 44. , 99.6])

In [ ]:
np.array([np.median(fig2u_all_snr),np.quantile(fig2u_all_snr,0.25),np.quantile(fig2u_all_snr,0.75)])
# array([ 7.3,  4.9, 10.6])

In [ ]:
np.set_printoptions(precision=2)

In [ ]:
np.array([np.median(fig2u_all_fr),np.quantile(fig2u_all_fr,0.25),np.quantile(fig2u_all_fr,0.75)])
# array([1.12, 0.56, 2.22])

In [ ]:
np.set_printoptions(precision=1)

In [ ]:
np.array([np.mean(fig2u_all_density),np.std(fig2u_all_density)])
# array([5.3, 3.9])

In [ ]:
np.array([np.mean(fig2u_all_spread),np.std(fig2u_all_spread)])
# array([2.8, 1.1])